In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
!pip install transformers

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
import logging
from huggingface_hub import notebook_login
from pathlib import Path

logging.disable(logging.WARNING)

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

## Waiting for approval to use meta Llama

In [5]:
#mn = "meta-llama/Llama-2-7b-hf"

In [6]:
# waiting for approval for meta-Llma from meta repo on Huggingface
#model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_8bit=True)

In [5]:
!pip install accelerate

In [6]:
!pip install auto-gptq
!pip install --upgrade optimum

In [10]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)


## LLM from stabilityAi
I can not load this model on this nb since i am using Google Colab free version the Disk size can not handle the model it since it fill the 78.2 GB allocated storage.
So be advised if you are experimenting with Colab

The GPTQ optimized version can still work using colab (N.B. you might need to Restart your nb after installing the required libraries above, but it works.

In [11]:
#model = AutoModelForCausalLM.from_pretrained("stabilityai/StableBeluga2", torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto")
#system_prompt = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal.\n\n"


In [12]:
MN_= "TheBloke/Llama-2-7b-Chat-GPTQ"
#mn = "stabilityai/StableBeluga2"

In [25]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

In [9]:
toks

{'input_ids': tensor([[    1,  5677,  6764, 17430,   338,   263, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [10]:
tokr.batch_decode(toks['input_ids'])

['<s> Jeremy Howard is a ']

In [22]:
%%time

res = model.generate(**toks.to("cuda"), max_new_tokens=15).to("cpu")
res

CPU times: user 929 ms, sys: 0 ns, total: 929 ms
Wall time: 1.81 s


tensor([[    1,  5677,  6764, 17430,   338,   263, 29871, 29941, 29945, 29899,
          6360, 29899,  1025,   767,   515,   278,  3303,  3900,  1058,   471,
         24383,   297]])

In [23]:
tokr.batch_decode(res)

['<s> Jeremy Howard is a 35-year-old man from the United States who was arrested in']

In [24]:
mn = 'TheBloke/Llama-2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)


In [28]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to("cpu")
res

RuntimeError: ignored

In [26]:
def gen(p,maxlen=15, sample=True):
  toks = tokr(p , return_tensors='pt')
  res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to("cpu")
  return tokr.batch_decode(res)

In [27]:
gen(prompt,50)

RuntimeError: ignored

In [29]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)
#model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_type=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [12]:
#sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [13]:
def mk_pmt(user, syst=sb_sys): return f"{syst}###user: {user}\n\n### Assistant:\n"

In [21]:
!ls ~/.cache/huggingface/hub

models--TheBloke--OpenOrca-Platypus2-13B-GPTQ  version.txt


In [22]:
ques = "Who is Jeremy Howard?"

In [21]:
gen(mk_pmt(ques),150)

['<s> ### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n###user: Who is Jeremy Howard?\n\n### Assistant:\n Jeremy Howard is an Australian entrepreneur, data scientist, and an artificial intelligence researcher. He co-founded the AI company, Enlitic, and is a co-founder and board member of the open-source machine learning software company, Fast.ai.</s>']

#### lets try with OpenOrca-Platypus2-13b-GPTQ

In [7]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [23]:
#def mk_oo_prmt(user): return f"### Instruction: {user}\n\n ###Response:\n"
def mk_oo_prompt(user): return f"### Instruction: {user}\n\n### Response:\n"


In [24]:
gen(mk_oo_prompt(ques), 150)

RuntimeError: ignored